In [16]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import optim




In [33]:
import numpy as np
import pandas as pd
from scipy.special import expit

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import tensor
import torch.optim as optim

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [7]:
data_train = np.load('data/fashion_train.npy', mmap_mode='r')
data_test = np.load('data/fashion_test.npy', mmap_mode='r')

In [8]:
X_train = data_train[:, :-1]
y_train = data_train[:, -1]
X_test = data_test[:, :-1]
y_test = data_test[:, -1]

In [42]:
class CNN(nn.Module):
    def __init__(self, **INPUT):

        # Inherit from nn module
        super(CNN, self).__init__()

        # Define activation function
        self.af = INPUT.get('af')

        # Define drop ratio
        self.dpr = INPUT.get('dpr')

        # Define neural network architecture
        self.nn = nn.Sequential(
                # C1 6@28x28
                nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
                nn.BatchNorm2d(6),
                self.af(),
                nn.AvgPool2d(kernel_size=2, stride=2),
                
                # C2: 16@10x10
                nn.Dropout(self.dpr),
                nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
                nn.BatchNorm2d(16),
                self.af(),
                nn.AvgPool2d(kernel_size=2, stride=2),
                
                # Apply flattening on the output
                nn.Flatten(),
                
                # Dense part
                # L1
                nn.Dropout(self.dpr),
                nn.Linear(16 * 5 * 5, 120),
                nn.BatchNorm1d(120),
                self.af(),
                
                # L2
                nn.Dropout(self.dpr),
                nn.Linear(120, 84),
                nn.BatchNorm1d(84),
                self.af(),
                
                # L3
                nn.Dropout(self.dpr),
                nn.Linear(84, 10))
        
        # Define batch size
        self.batch_size = INPUT.get('batch_size')

        # Define datasets
        self.training_data = DataLoader(
            INPUT.get('trd'), batch_size=self.batch_size, shuffle=True, num_workers=1)
        self.validation_data = DataLoader(
            INPUT.get('vd'), batch_size=self.batch_size, shuffle=True, num_workers=1)
        self.test_data = DataLoader(
            INPUT.get('ted'), batch_size=self.batch_size, shuffle=True, num_workers=1)

        # Define loss function
        self.loss_fn = INPUT.get('loss_fn')

        # Define learning rate
        self.lr = INPUT.get('lr')

        # Define numper of epochs
        self.epochs = INPUT.get('epochs')

        # Define optimizer
        self.optimizer = INPUT.get('optim')(self.parameters(), lr=self.lr)

        # Save training progress
        self.loss_history = []
        self.acc_history = []
    
    def forward(self, x):
        logits = self.nn(x)
        return logits
    
    def train_loop(self):
        
        size = len(self.training_data.dataset)
        for batch, (X, y) in enumerate(self.training_data):

            # Compute prediction and loss
            pred = self.forward(X)
            loss = self.loss_fn(pred, y)

            # Backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    def val_loop(self):
        size = len(self.validation_data.dataset)
        num_batches = len(self.validation_data)
        test_loss, correct = 0, 0

        with torch.no_grad():
            for X, y in self.validation_data:
                pred = self.forward(X)
                test_loss += self.loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
        test_loss /= num_batches
        correct /= size

        print(
            f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

        # Save it to history
        self.acc_history.append(correct)
        self.loss_history.append(test_loss)

    def visualize(self):
        x = [i for i in range(self.epochs)]
        y1 = self.acc_history
        y2 = self.loss_history
        plt.plot(x, y1)
        plt.plot(x, y2)
        plt.show()

    def fit(self):
        for t in range(self.epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            self.train_loop()
            self.val_loop()
        print("Done!")

    def predict(self, x):
        logits = self.forward(x)
        softmax = nn.Softmax(dim=1)
        return softmax(logits).argmax(1)

    def test(self):
        # Get data
        X, y = next(iter(self.test_data))

        # Predict values
        y_hat = self.predict(X)

        print("Accuracy score for test data")
        print("-"*60)
        print(f"Acc: {accuracy_score(y, y_hat)*100} %")
        print()
        print("Confusion matrix for test data")
        print("-"*60)
        print(confusion_matrix(y, y_hat))

In [43]:


# Split training into  training and validation
g_cpu = torch.Generator()
g_cpu.manual_seed(3)
training_data, val_data = torch.utils.data.random_split(data_train, [50000, 10000])

# Initialize model
INPUT = {
    'batch_size': 100,
    'trd': training_data,
    'vd': val_data,
    #'ted': test_data,
    'loss_fn': nn.CrossEntropyLoss(),
    'lr': 1e-1,
    'epochs': 5,
    'af': nn.Sigmoid,
    'optim': torch.optim.Adam,
    'dpr': 1e-3

}
model = CNN(**INPUT)

# Train model
model.fit()

TypeError: object of type 'NoneType' has no len()